# 3조_임재원_스팸문자분류
---
### 목차
- 데이터 전처리
- 모델 생성
-

In [117]:
# 모듈 로딩
import numpy as np
import pandas as pd
import re
from konlpy.tag import Kkma, Komoran, Okt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

# [1] 데이터 로딩 및 확인

In [118]:
spam_1 = pd.read_csv(r'./data/spam/스팸문자내역1.csv', encoding='utf-8')
spam_2 = pd.read_csv(r'./data/spam/스팸문자내역2.csv', encoding='utf-8')
ham_1 = pd.read_csv(r'./data/ham/서울시 구청 재난문자 발송현황.csv', encoding='cp949')
ham_2 = pd.read_csv(r'./data/ham/환경산업기술원 챗봇 대화세트 구성.csv', encoding='cp949')

In [119]:
spam_1.head(5)

,index,text
0,1,[국제발신]ifg@? 명절 *만 이벤진행ifg@ifg@? 최고의 메리트ifg@ifg...
1,2,[국외발신]ifg@●빅토리●ifg@ifg@퇴근시간**분전ifg@뒷면이보일정도로ifg...
2,3,[국제발신]ifg@오빠안녕하세요ifg@관리드렸던지수에요ifg@이번에도ifg@확실하게...
3,4,[국제발신]ifg@ifg@VERTifg@ifg@처음 **%ifg@삼+삼ifg@**+...
4,5,[국외발신]ifg@?그리스?ifg@이붼츄가넘쳐유~ifg@커피/치킨???ifg@항상감...


In [120]:
ham_1.head(5)

,연,월,일,시,분,지역,송출내용
0,2021,2,1,9,46,강북구,"[강북구청]확진자5명(620~624번)발생, 동선( www.gangbuk.go.kr..."
1,2021,2,1,10,0,노원구,[노원구청] 오늘(2.1.) 오전 확진자 2명 발생(1047~1048번). 역학조사...
2,2021,2,1,10,0,광진구,[광진구청] 타지역(중랑구)확진자 관내 동선 알림.상호 공개된 업소 홈페이지( ha...
3,2021,2,1,10,9,도봉구,[도봉구청] 2.1(월) 확진자 1명 발생(748번). 역학조사결과 이동동선 및 조...
4,2021,2,1,10,13,성북구,[성북구청] 1020~1023번(4명) 확진자 발생. 거주지 방역 완료. 자세한 사...


# [2] 데이터 전처리 및 병합
- text, target의 열을 가진 새로운 데이터프레임 생성
- 불러온 데이터프레임에서 문자내역을 text열에 담고, target에 spam일 경우 1 / ham일 경우 0 지정
### But, 스팸문자가 약 17000개, 정상문자가 2425개로 데이터의 불균형이 심각함
- 스팸문자의 경우 비슷비슷한 경우가 많기때문에 스팸문자의 비율을 언더샘플링하여 정상문자와 비슷한 수로 맞춰줌

In [121]:
# 새로운 데이터프레임 생성
text_data = pd.DataFrame(columns=[['text', 'target']])

# spam, ham 문자메세지 target 생성
spam_1['target'] = 1
spam_2['target'] = 1
ham_1['target'] = 0
ham_2['target'] = 0

# concat준비
spam_1.drop(['index'], axis=1, inplace=True)
spam_2.drop(['index'], axis=1, inplace=True)
ham_1.drop(ham_1.columns[0:6], axis=1, inplace=True)
ham_2.drop(ham_2.columns[0:4], axis=1, inplace=True)
ham_2.drop(ham_2.columns[1:7], axis=1, inplace=True)
ham_1.rename(columns={'송출내용': 'text'}, inplace=True)
ham_2.rename(columns={'답변내용(4,000자 이내) - 필수': 'text'}, inplace=True)
spam_1 = spam_1.sample(n=1200, random_state=42)
spam_2 = spam_2.sample(n=1200, random_state=42)

# text_data에 병합
text_data = pd.concat([spam_1, spam_2, ham_1, ham_2], ignore_index=True)

In [122]:
# 병합된 데이터 확인
text_data

,text,target
0,[국제발신]ifg@화요일지원금ifg@도착하였습니다ifg@코::드 - ***ifg@서...,1
1,[국제발신]ifg@체리#**★타ifg@추억의당구장ifg@원바#투바#올과일ifg@**...,1
2,"[국외발신]ifg@새해엔좋은일만가득하세요ifg@적립 ***,***원ifg@ifg@l...",1
3,[국외발신]ifg@카】강원ifg@ⓖ】마카오보다ifg@노】여기로오세요ifg@올구조대'...,1
4,[국제발신]ifg@심심할때오세요ifg@시간잘~갑니다ifg@**.***원도줍니다ifg...,1
...,...,...
4820,제품사후관리실에서는 환경표지등의 인증을 받지 아니하고 환경표지 등 또는 이와 유사한...,0
4821,"가능합니다. 다만, 기 인증 제품과 환경성 및 품질에 변동사항이 없다면(원료사용 내...",0
4822,환경표지 홈페이지(http://el.keiti.re.kr) 홈 → 정보마당 → 서식...,0
4823,"파생제품 등록의 경우 현장심사나 시험의뢰를 진행하지 않으며, 서류심사를 통해 진행됩니다.",0


In [123]:
# value_counts()로 스팸, 정상문자의 수 확인
text_data['target'].value_counts()

0    2425
1    2400
Name: target, dtype: int64

# [3] 문자내용 전처리
- 영어와 특수문자 제거
- konlpy를 통한 텍스트 토큰화
- 불필요한 단어들 제거
- CountVectorizer를 통한 단어간 거리 수치화

In [124]:
# 영어와 특수문자 제거
text_data['text'] = [re.sub('[^0-9가-힣]', '', s) for s in text_data['text']]

In [125]:
# konlpy를 통한 텍스트 토큰화
tokenizer = Okt()
train_divide = [(tokenizer.pos(x), y) for x, y in zip(text_data['text'], text_data['target'])]

# 데이터 확인
pd.DataFrame(np.array(train_divide))

C:\ProgramData\Anaconda3\envs\EV_PY37\lib\site-packages\ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


,0,1
0,"[(국제, Noun), (발신, Noun), (화요일, Noun), (지, Josa...",1
1,"[(국제, Noun), (발, Noun), (신체, Noun), (리타, Noun)...",1
2,"[(국외, Noun), (발신, Noun), (새해, Noun), (엔, Josa)...",1
3,"[(국외, Noun), (발신, Noun), (카, Noun), (강원, Noun)...",1
4,"[(국제, Noun), (발신, Noun), (심심할, Adjective), (때,...",1
...,...,...
4820,"[(제품, Noun), (사후관리, Noun), (실, Noun), (에서는, Jo...",0
4821,"[(가능합니다다만, Adjective), (기, Modifier), (인증, Nou...",0
4822,"[(환경, Noun), (표지, Noun), (홈페이지, Noun), (홈, Nou...",0
4823,"[(파생, Noun), (제품, Noun), (등록, Noun), (의, Josa)...",0


In [126]:
# 불필요한 단어들
stopwords = ['을', '를', '이', '가', '은', '는', '국제', '발신']


# 단어 제거 함수 생성
def get_couple(words):
    global stopwords
    words = [x for x in words if x[0] not in stopwords]
    lex_text = len(words)
    for j in range(lex_text - 1):
        yield words[j][0], words[j + 1][0]


def get_couple2(words):
    global stopwords
    words = [x for x in words if x[0] not in stopwords]
    lex_text = len(words)
    for j in range(lex_text - 1):
        yield words[j][0], words[j + 1][0]

In [127]:
# 불필요한 단어 제거 후 text와 target 리스트 재생성
text, target = [], []
for lwords in train_divide:
    target.append(lwords[1])
    temp = []
    for x, y in get_couple(lwords[0]):
        temp.append("{}.{}".format(x, y))
    text.append(" ".join(temp))

In [128]:
# CountVectorizer를 통한 단어간 거리 수치화
vec = CountVectorizer()
vec.fit(text)
vec_text = vec.transform(text).toarray()

# [4] 학습 / 테스트 데이터 분리

In [129]:
train_X, test_X, train_y, test_y = train_test_split(vec_text, target, test_size=0.2, random_state=42)

In [130]:
m1 = MultinomialNB()
m1.fit(train_X, train_y)
m1.score(train_X, train_y)
m1.score(test_X, test_y)

0.9740932642487047

In [137]:
text = '히어로의 과학 설명회 하루종일 설명할 수도 있어'
text = [re.sub('[^0-9가-힣]', '', s) for s in text]
text = ''.join(text)
tokenizer = Okt()
temp = [(tokenizer.pos(text), '0')]
train_divide = train_divide + temp

# 불필요한 단어 제거 후 text와 target 리스트 재생성
text, target = [], []
for lwords in train_divide:
    target.append(lwords[1])
    temp = []
    for x, y in get_couple(lwords[0]):
        temp.append("{}.{}".format(x, y))
    text.append(" ".join(temp))

# CountVectorizer를 통한 단어간 거리 수치화
vec = CountVectorizer()
vec.fit(text)
vec_text = vec.transform(text).toarray()

In [138]:
m1.predict([vec_text[-1]])

ValueError: X has 8389 features, but MultinomialNB is expecting 8386 features as input.